In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange, repeat
import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from utility import listFiles
from scipy.spatial.transform import Rotation as R
from numpy import cos, sin
import torch.nn as nn
### fucking around
from pretrain import loadDataFromRegisteredSlam, cart2sph, preProcess
from train import getSamples, getUpSamples, getSpacing

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [50]:
def getSpacing(num_points, num_bins):
    """return a [num_points*num_bins, 1] pytorch tensor
    
    """
    # TODO: add hyperparameter for tuning "slope" of inverse sigmoid function
    # create a list of magnitudes with even spacing from 0 to 1
    t = torch.linspace(0,1, num_bins).expand(num_points, num_bins)  # [batch_size, num_bins//2]
    
    # preterb the spacing
    mid = (t[:, :-1] + t[:, 1:]) / 2.
    lower = torch.cat((t[:, :1], mid), -1)
    upper = torch.cat((mid, t[:, -1:]), -1)
    u = torch.rand(t.shape)
    t = lower + (upper - lower) * u  # [batch_size, nb_bins//2]
    t[:,0] = 1e-4
    t[:,-1] = 0.995
    # apply inverse sigmoid function to even spacing
    t = rearrange(t, 'a b -> (a b) 1')  # [num_bins, batch_size]  transpose for multiplication broadcast
    t = torch.log(t / (torch.abs(1 - t)+ 1e-8))  
    return t

In [58]:
x = getSpacing(10000,100)
# max(x)


torch.Size([10000, 100])
tensor([[1.0000e-04, 6.1801e-03, 1.8660e-02,  ..., 9.7502e-01, 9.9039e-01,
         9.9500e-01],
        [1.0000e-04, 1.0230e-02, 1.5639e-02,  ..., 9.8180e-01, 9.8749e-01,
         9.9500e-01],
        [1.0000e-04, 1.4302e-02, 2.1242e-02,  ..., 9.8188e-01, 9.8897e-01,
         9.9500e-01],
        ...,
        [1.0000e-04, 9.2310e-03, 1.5599e-02,  ..., 9.7831e-01, 9.9397e-01,
         9.9500e-01],
        [1.0000e-04, 1.5042e-02, 2.2134e-02,  ..., 9.8088e-01, 9.8666e-01,
         9.9500e-01],
        [1.0000e-04, 1.1250e-02, 2.4384e-02,  ..., 9.8266e-01, 9.9365e-01,
         9.9500e-01]])


In [4]:
name = r'round_plant2'
path = r'datasets/registered/' + name + r'.json'
data = loadDataFromRegisteredSlam(path)
training_data = preProcess(data)
np.random.shuffle(training_data)

Preparing data ... (1150/1187)

In [4]:
num_points = 1024
ground_truth_distance = training_data[:num_points,0]
angles = training_data[:num_points,1:3]
origins = training_data[:num_points,3:6]

In [6]:
gt_dist_tensor = torch.tensor(ground_truth_distance)
ang_tensor = torch.tensor(angles)
origins_tensor = torch.tensor(origins)

In [9]:
sample_pos, sample_ang, sample_org = getSamples(origins_tensor, ang_tensor, gt_dist_tensor, num_bins=100)
upsample_pos, upsample_ang, upsample_gt_distance = getUpSamples(origins_tensor, ang_tensor, gt_dist_tensor, num_rolls=0)

In [11]:
# tile distances
gt_distance_tiled = torch.tensor(repeat(ground_truth_distance, 'b -> (b n) 1', n=100))

# stack the upsampled position to sampled positions
pos = (torch.vstack((sample_pos, upsample_pos)))
ang = (torch.vstack((sample_ang, upsample_ang)))
gt_dis = (torch.vstack((gt_distance_tiled,upsample_gt_distance)))
org = (torch.vstack((sample_org, upsample_pos)))


In [15]:
actual_value_sigmoided = (getTargetValues(pos, gt_dis, org)).to(dtype = torch.float32)

/tmp/ipykernel_45895/3882400413.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  temp = torch.tensor((sample_positions - origins)**2)


In [17]:
max(actual_value_sigmoided)

tensor([0.9998])

In [12]:
# returns pytorch tensor of sigmoid of projected SDF
def getTargetValues(sample_positions, gt_distance, origins, num_bins=100):
    # calculate distance from sample_position
    temp = torch.tensor((sample_positions - origins)**2)
    pos_distance = torch.sqrt(torch.sum(temp, dim=1, keepdim=True))
    
    # find the "projected" value
    sigmoid = nn.Sigmoid()
    values = sigmoid(-(pos_distance - gt_distance))
    return values

In [162]:
# Convert the numpy array to an Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(sample_pos)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])